In [ ]:
import librosa
from librosa import display
from IPython.display import Audio
import matplotlib.pyplot as plt
import scipy
import numpy as np
import math
import pandas as pd

In [ ]:
audio_with_plane, sampling_rate_with_plane = librosa.load("../sample_audio_with_plane.mp3")

len(audio_with_plane), sampling_rate_with_plane, len(audio_with_plane)/sampling_rate_with_plane

In [ ]:
audio_without_plane, sampling_rate_without_plane = librosa.load("../sample_audio_without_plane.mp3")

len(audio_without_plane), sampling_rate_without_plane, len(audio_without_plane)/sampling_rate_without_plane

In [ ]:
max_freq = 0
min_freq = 10000

n = len(audio_with_plane)
T = 1/sampling_rate_with_plane
xf = np.linspace(0.0, 1.0/(2.0*T), n//2)

max_freq = max(max_freq, max(list(xf)))
min_freq = min(min_freq, min(list(xf)))

n = len(audio_without_plane)
T = 1/sampling_rate_without_plane
xf = np.linspace(0.0, 1.0/(2.0*T), n//2)

max_freq = max(max_freq, max(list(xf)))
min_freq = min(min_freq, min(list(xf)))


In [ ]:
def fft_plot(audio, sampling_rate):
  n = len(audio)
  T = 1/sampling_rate
  yf = scipy.fft.fft(audio)
  xf = np.linspace(0.0, 1.0/(2.0*T), n//2)
  fig, ax = plt.subplots()
  ax.plot(xf, 2.0/n * np.abs(yf[:n//2]))
  plt.grid()
  plt.xlabel("Frequency -->")
  plt.ylabel("Magnitude -->")
  return plt.show()

In [ ]:
fft_plot(audio_with_plane, sampling_rate_with_plane)

In [ ]:
fft_plot(audio_without_plane, sampling_rate_without_plane)

In [ ]:
def features_12(audio, sampling_rate):
  n = len(audio)
  T = 1/sampling_rate
  yf = scipy.fft.fft(audio)
  xf = list(np.linspace(0.0, 1.0/(2.0*T), n//2))
  yf = list(2.0/n * np.abs(yf[:n//2]))

  zipped_pairs = zip(yf, xf)
  sorted_zp = sorted(zipped_pairs)
  frequencies = [x for _, x in sorted_zp]
  magnitudes = [y for y, _ in sorted_zp]

  # max_freq = max(frequencies)
  # min_freq = min(frequencies)
  sum_features = [0 for i in range(12)]
  cnt_features = [0 for i in range(12)]

  for i in range(len(frequencies)):
    index = min(int(12*((frequencies[i]-min_freq)/math.ceil(max_freq-min_freq))), 11)
    cnt_features[index] += 1
    sum_features[index] += magnitudes[i]

  features = [0]*12
  for i in range(12):
    features[i] = sum_features[i]/max(1, cnt_features[i])

  return features

In [ ]:
features_with_plane = []
features_without_plane = []

for i in range(60):
  audio, sr = librosa.load("../Plane/Audio Intervals with plane/audio_interval_with_plane_" + str(i+1) + ".mp3")
  features_with_plane.append(features_12(audio, sr))

  audio, sr = librosa.load("../Plane/Audio Intervals without plane/audio_interval_without_plane_" + str(i+1) + ".mp3")
  features_without_plane.append(features_12(audio, sr))

In [ ]:
df_with_plane = pd.DataFrame(np.array(features_with_plane))
df_with_plane.head()

In [ ]:
df_without_plane = pd.DataFrame(np.array(features_without_plane))
df_without_plane.head()

In [ ]:
df_with_plane.to_csv("../Plane/CSV/audio_with_plane.csv")
df_without_plane.to_csv("../Plane/CSV/audio_without_plane.csv")